In [1]:
import pandas as pd
import numpy as np
import glob
import re
import cv2
import os

In [4]:
# 設定資料夾路徑和檔案模式
folder_path = "data/"  # 設定資料夾路徑
image_pattern_front = "front_video_swing_plane_*.jpg"
image_pattern_side = "side_video_swing_plane_*.jpg"
csv_pattern_front = "front_video.csv"
csv_pattern_side = "side_video.csv"

combined_path = "data/combined/"

# 定義點、線的顏色及粗細
point_color = (0, 0, 255)  # 紅色點
line_color = (0, 255, 0)   # 綠色線段
text_color = (255, 255, 255) # 白色文字
point_radius = 5
point_thickness = -1  # 填滿圓圈
line_thickness = 2
font_scale = 0.5
font_thickness = 1

# 定義骨架的連接順序
skeleton_connections = [
    (0, 1), (1, 2), (0, 2),
    (1, 3),(2, 4),
    (3, 5),(4, 6),(5, 6),
    (5, 7),(6, 8),
    (7, 9),(8, 10),
    (5, 11),(6, 12),(11, 12),
    (12, 14),(14, 16),
    (11, 13),(13, 15)
]

In [5]:
def extract_frame_number(filename):
    """從檔名中提取最後的編號"""
    match = re.search(r"(\d+)\_plane.jpg$", filename)
    return int(match.group(1)) if match else None

def load_images_and_csv(image_pattern, csv_file):
    """讀取圖片和 CSV 資料，並根據檔名編號標記每張圖片"""
    # 讀取圖片檔案
    image_paths = sorted(glob.glob(folder_path + image_pattern))
    # 讀取 CSV
    if not os.path.isfile(folder_path + csv_file):
        print(f"Error: File not found - {folder_path + csv_file}")
        return None
    df = pd.read_csv(folder_path + csv_file, index_col=0)  # 假設第一列為索引（即編號）

    marked_images = []
    for image_path in image_paths:
        # 提取圖片編號
        frame_number = extract_frame_number(image_path)
        if frame_number is None or frame_number not in df.index:
            print(f"無法找到對應的數據行: {image_path}")
            continue

        # 提取每個點的 (x, y) 坐標
        points = []
        for i in range(17):  # 17 個點
            x_col = f'pose{i}_x'
            y_col = f'pose{i}_y'
            if x_col in df.columns and y_col in df.columns:
                x, y = df.at[frame_number, x_col], df.at[frame_number, y_col]
                points.append((x, y))

        # 讀取圖片
        image = cv2.imread(image_path)
        original_height, original_width = image.shape[:2]
        # 調整圖片大小為 608x1080
        target_size = (608, 1080)
        image = cv2.resize(image, target_size)

        # 計算縮放比例
        scale_x = target_size[0] / original_width
        scale_y = target_size[1] / original_height
        scaled_points = [(int(x * scale_x), int(y * scale_y)) for (x, y) in points]

        # 繪製點位和標記點的編號
        for idx, (x, y) in enumerate(scaled_points):
            # 繪製點
            cv2.circle(image, (x, y), point_radius, point_color, point_thickness)
            # 標記點的編號
            cv2.putText(image, str(idx), (x + 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                        font_scale, text_color, font_thickness)

        # 繪製骨架連接線
        for start_idx, end_idx in skeleton_connections:
            start_point = scaled_points[start_idx]
            end_point = scaled_points[end_idx]
            cv2.line(image, start_point, end_point, line_color, line_thickness)

        # 將標記過的圖片加入列表
        marked_images.append(image)
    return marked_images



In [6]:
for subdir in os.listdir(folder_path):
    subdir_path = os.path.join(folder_path, subdir)
    if os.path.isdir(subdir_path):
        front_images = load_images_and_csv(os.path.join(subdir, image_pattern_front), os.path.join(subdir, csv_pattern_front))
        side_images = load_images_and_csv(os.path.join(subdir, image_pattern_side), os.path.join(subdir, csv_pattern_side))


        if front_images and side_images:
            front_combined = np.hstack(front_images)
            side_combined = np.hstack(side_images)
            final_combined = np.vstack([front_combined, side_combined])
            output_path = os.path.join(combined_path, f"combined_{subdir}.jpg")
            cv2.imwrite(output_path, final_combined)
            print(f"最終合併的圖片已儲存到: {output_path}")

最終合併的圖片已儲存到: data/combined/combined_186382.jpg
最終合併的圖片已儲存到: data/combined/combined_186387.jpg
Error: File not found - data/186406\front_video.csv
Error: File not found - data/186406\side_video.csv
最終合併的圖片已儲存到: data/combined/combined_186410.jpg
最終合併的圖片已儲存到: data/combined/combined_186416.jpg
最終合併的圖片已儲存到: data/combined/combined_189863.jpg
最終合併的圖片已儲存到: data/combined/combined_198514.jpg
最終合併的圖片已儲存到: data/combined/combined_199353.jpg
最終合併的圖片已儲存到: data/combined/combined_199453.jpg
Error: File not found - data/combined\front_video.csv
Error: File not found - data/combined\side_video.csv
Error: File not found - data/not_complete\front_video.csv
Error: File not found - data/not_complete\side_video.csv
